# 16's place solution

This kenel based on https://www.kaggle.com/kneroma/clean-fast-simple-bird-identifier-inference My model's weights trained with this code https://www.kaggle.com/kneroma/clean-fast-simple-bird-identifier-training-colab

In [ ]:
!nvidia-smi

In [ ]:
try:
    import resnest
except ModuleNotFoundError:
    !pip install -q "../input/resnest50-fast-package/resnest-0.0.6b20200701/resnest"

In [ ]:
!pip install ../input/timm-pytorch-image-models/pytorch-image-models-master/
!pip install ../input/torchlibrosa/torchlibrosa-0.0.5-py3-none-any.whl

In [ ]:
!pip install ../input/pytorchlightning/tensorboard-2.2.0-py3-none-any.whl ../input/pytorchlightning/pytorch_lightning-0.9.0-py3-none-any.whl

In [ ]:
!pip install ../input/torchaudiomentations070/torch_audiomentations-0.7.0-py3-none-any.whl 

In [ ]:
import numpy as np
import librosa
import soundfile as sf
import pandas as pd
import cv2
from pathlib import Path
import re

import torch
from torch import nn, optim
from  torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F


import torchaudio

from torchlibrosa.stft import LogmelFilterBank, Spectrogram
from torchlibrosa.augmentation import SpecAugmentation

import timm
from contextlib import contextmanager
from typing import Optional
import warnings

import audioread
import logging
import random

import os
import json

from tqdm.notebook import tqdm

import time
from resnest.torch import resnest50

from albumentations.core.transforms_interface import ImageOnlyTransform

import joblib

import gc

from sklearn.metrics import label_ranking_average_precision_score

import pytorch_lightning as pl

import torch_audiomentations as AA

In [ ]:
torch.__version__

# Configs

In [ ]:
NUM_CLASSES = 397
SR = 32_000
DURATION = 5
THRESH = 0.25


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE:", DEVICE)

TEST_AUDIO_ROOT = Path("../input/birdclef-2021/test_soundscapes")
SAMPLE_SUB_PATH = "../input/birdclef-2021/sample_submission.csv"
PUBLIC = False
#TARGET_PATH = None
    
if not len(list(TEST_AUDIO_ROOT.glob("*.ogg"))):
    TEST_AUDIO_ROOT = Path("../input/birdclef-2021/train_soundscapes")
    SAMPLE_SUB_PATH = None
    # SAMPLE_SUB_PATH = "../input/birdclef-2021/sample_submission.csv"
    PUBLIC = True
    
TARGET_PATH = Path("../input/birdclef-2021/train_soundscape_labels.csv")

# Data

## Transform audio ogg to melspec npy 

In [ ]:
data = pd.DataFrame(
     [(path.stem, *path.stem.split("_"), path) for path in TEST_AUDIO_ROOT.glob("*.ogg")],
    columns = ["filename", "id", "site", "date", "filepath"]
)
print(data.shape)
data.head()

In [ ]:
public_data = pd.DataFrame(
     [(path.stem, *path.stem.split("_"), path) for path in Path("../input/birdclef-2021/train_soundscapes").glob("*.ogg")],
    columns = ["filename", "id", "site", "date", "filepath"]
)
print(data.shape)
data.head()

In [ ]:
!rm -rf test_data_1 
!mkdir test_data_1 
TARGET_AUDIO_1 = Path("test_data_1")


In [ ]:
class MelSpecComputer:
    def __init__(self, sr, n_mels, fmin=0, fmax=16000, **kwargs):
        self.sr = sr
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax
        kwargs["n_fft"] = kwargs.get("n_fft", self.sr//10)
        kwargs["hop_length"] = kwargs.get("hop_length", self.sr//(10*4))
        self.kwargs = kwargs

    def __call__(self, y):

        melspec = librosa.feature.melspectrogram(
            y, sr=self.sr, n_mels=self.n_mels, fmin=self.fmin, fmax=self.fmax, **self.kwargs,
        )

        melspec = librosa.power_to_db(melspec).astype(np.float32)
        return melspec

In [ ]:
import math
def load_audio(record, sr=32000, root=Path("./")):
    waveform, sample_rate = torchaudio.load(
        root.joinpath(record).with_suffix(".ogg").as_posix(),
    )
    if sample_rate != sr:
        y = torchaudio.transforms.Resample(sample_rate, sr)(waveform)
        return y
    else:
        return waveform

sound_len = 5 * 32000

do_melspec_1 = MelSpecComputer(sr=32000, n_mels=128)

#Тестовый файл нарезается на фрагменты, длинною sound_len, 
def load_and_save_test(row):
   # try:
        y = load_audio(row["filepath"], 32000)
        y = y.squeeze()
        len_y = len(y)
        #print(row["t_min"], row["t_max"])
        #print(len_y)

        for i in range(math.ceil(len(y) / sound_len)):
            t_min = max(i * sound_len - 32000 , 0)
            t_max = min((i +1) * sound_len + 32000, len_y)
            #print(t_min, t_max)
            labl = np.array(y[int(t_min):int(t_max)])#.view(1,-1)
            if len(labl) < (7 * 32000):
                labl = np.hstack([labl, labl])[:(7 * 32000)]
            #print(len(labl))
            
            np.save(TARGET_AUDIO_1/("_".join(row["filename"].split("_")[:2] + [str((i+1)*5)])+".npy"), do_melspec_1(labl), allow_pickle=True)
            

for n, row in tqdm(data.iterrows(), total=data.shape[0]):
    load_and_save_test(row)

if PUBLIC:
    pass
else:
    for n, row in tqdm(public_data.iterrows(), total=public_data.shape[0]):
        load_and_save_test(row)

In [ ]:
# import IPython
# IPython.display.Audio("test_data/11254_COR_10.ogg")

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
plt.imshow(np.load("test_data_1/11254_COR_10.npy"))

## Use data in models

In [ ]:
def normalize(X, eps=1e-6, mean=None, std=None):
    mean = mean or X.mean()
    std = std or X.std()
    X = (X - mean) / (std + eps)
    
    _min, _max = X.min(), X.max()

    if (_max - _min) > eps:
        V = np.clip(X, _min, _max)
        V = 255 * (V - _min) / (_max - _min)
        V = V.astype(np.uint8)
    else:
        V = np.zeros_like(X, dtype=np.uint8)

    return V


def mono_to_color(image):
    image = image.astype("float32", copy=False) / 255.0
    image = np.stack([image, image, image])
    return image

def crop_or_pad(y, length):
    if len(y) < length:
        y = np.concatenate([y, length - np.zeros(len(y))])
    elif len(y) > length:
        y = y[:length]
    return y

In [ ]:
apply_augmentation = AA.Compose(
    transforms=[
        AA.Gain(
            min_gain_in_db=-5.0,
            max_gain_in_db=5.0,
            p=0.5,
        ),
        AA.AddColoredNoise(p = 0.9, min_f_decay=0,  max_f_decay=1, 
                           min_snr_in_db=5,  max_snr_in_db = 20, sample_rate = 32000), # white, pink noise
        AA.Shift(p=0.2, min_shift=-0.2, max_shift=0.2, sample_rate = 32000),
    ]
)

In [ ]:
class BirdCLEFDataset(Dataset):
    def __init__(self, data, sr=SR, n_mels=128, fmin=0, fmax=None, duration=DURATION, step=None, res_type="kaiser_fast", resample=True, transform = False):
        
        self.data = data
        
        self.sr = sr
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax or self.sr//2

        self.duration = duration
        self.audio_length = self.duration*self.sr
        self.step = step or self.audio_length
        
        self.res_type = res_type
        self.resample = resample
        self.transform = transform

        self.mel_spec_computer = MelSpecComputer(sr=self.sr, n_mels=self.n_mels, fmin=self.fmin,
                                                 fmax=self.fmax)
    def __len__(self):
        return len(self.data)
    

    
    def audio_to_image(self, audio):
        melspec = self.mel_spec_computer(audio) 
        image = mono_to_color(melspec)
        image = self.normalize(image)
        return image

    def read_file(self, filepath):
        audio, orig_sr = sf.read(filepath, dtype="float32")

        if self.resample and orig_sr != self.sr:
            audio = lb.resample(audio, orig_sr, self.sr, res_type=self.res_type)
          
        audios = []
        for i in range(self.audio_length, len(audio) + self.step, self.step):
            start = max(0, i - self.audio_length)
            end = start + self.audio_length
            audios.append(audio[start:end])
            
        if len(audios[-1]) < self.audio_length:
            audios = audios[:-1]
            
        images = [self.audio_to_image(audio) for audio in audios]
        images = np.stack(images)
        
        return images
    def read_ready_file(self, filename):
        audios = []
        i = 0
        fname = "_".join(filename.split("_")[:2] + [str((i+1)*5)])+".npy"
        all_files = os.listdir(TARGET_AUDIO_1)
        while fname in all_files:
            melspec = np.load(TARGET_AUDIO_1/fname)
            image = np.nan_to_num(normalize(melspec)) 
            image = mono_to_color(image)
            if self.transform:
                image = apply_augmentation(torch.Tensor(image))
 
            #print(fname)
            audios.append(image)
            
            i+=1
            fname = "_".join(filename.split("_")[:2] + [str((i+1)*5)])+".npy"
        return np.stack(audios)
            
    def __getitem__(self, idx):
        return self.read_ready_file(self.data.loc[idx, "filename"])

In [ ]:
df_train = pd.read_csv("../input/birdclef-2021/train_metadata.csv")

LABEL_IDS = {label: label_id for label_id,label in enumerate(sorted(df_train["primary_label"].unique()))}
INV_LABEL_IDS = {val: key for key,val in LABEL_IDS.items()}

# Inference

In [ ]:
test_data = BirdCLEFDataset(data=data)
len(test_data), test_data[3].shape

In [ ]:
def load_net(checkpoint_path, num_classes=NUM_CLASSES):
    #net = resnest50(pretrained=False)
    net = timm.create_model('resnest50d', pretrained = False)
    #print(net)
    net.fc = nn.Linear(net.fc.in_features, num_classes)
    dummy_device = torch.device("cpu")
    d = torch.load(checkpoint_path, map_location=dummy_device)
    for key in list(d.keys()):
        d[key.replace("model.", "")] = d.pop(key)
    net.load_state_dict(d)
    net = net.to(DEVICE)
    net = net.eval()
    return net

def load_net2(checkpoint_path, num_classes=NUM_CLASSES):
    #net = resnest50(pretrained=False)
    net = timm.create_model('resnest50d_4s2x40d', pretrained = False)
    #print(net)
    net.fc = nn.Linear(net.fc.in_features, num_classes)
    dummy_device = torch.device("cpu")
    d = torch.load(checkpoint_path, map_location=dummy_device)['state_dict']
    for key in list(d.keys()):
        d[key.replace("model.", "")] = d.pop(key)
    net.load_state_dict(d)
    net = net.to(DEVICE)
    net = net.eval()
    return net

In [ ]:

nets = [
    load_net(Path("../input/kkiller-birdclef-models-public/birdclef_resnest50_fold0_epoch_10_f1_val_06471_20210417161101.pth").as_posix()),
    #Path("../input/hpa-mw2/birdclef_resnest50d_fold0_epoch_09_f1_val_07535_20210518114548.pth"),
    #Path("../input/hpa-mw2/resnest50d_4s2x40d-epoch4-val_loss0.023-val_f10.619.ckpt")
    load_net2(Path("../input/birdclef-for-nadya/resnest50d_4s2x40d-epoch11-val_loss0.022-val_f10.730.ckpt").as_posix())
]



In [ ]:
@torch.no_grad()
def get_thresh_preds(out, thresh=None):
    thresh = thresh or THRESH
    o = (-out).argsort(1)
    npreds = (out > thresh).sum(1)
    preds = []
    for oo, npred in zip(o, npreds):
        preds.append(oo[:npred].cpu().numpy().tolist())
    return preds

In [ ]:
def get_bird_names(preds):
    bird_names = []
    for pred in preds:
        if not pred:
            bird_names.append("nocall")
        else:
            bird_names.append(" ".join([INV_LABEL_IDS[bird_id] for bird_id in pred]))
    return bird_names

In [ ]:
def predict(nets, test_data, names=True):
    preds = []
    with torch.no_grad():
        for idx in  tqdm(list(range(len(test_data)))):
            xb = torch.from_numpy(test_data[idx]).to(DEVICE)
            pred = 0.
            for net in nets:
                o = net(xb)
                o = torch.sigmoid(o)

                pred += o

            pred /= len(nets)
            
            if names:
                pred = get_bird_names(get_thresh_preds(pred))

            preds.append(pred)
    return preds

In [ ]:
def preds_as_df(data, preds):
    sub = {
        "row_id": [],
        "birds": [],
    }
    
    for row, pred in zip(data.itertuples(False), preds):
        row_id = [f"{row.id}_{row.site}_{5*i}" for i in range(1, len(pred)+1)]
        sub["birds"] += pred
        sub["row_id"] += row_id
        
    sub = pd.DataFrame(sub)
    
    if SAMPLE_SUB_PATH:
        sample_sub = pd.read_csv(SAMPLE_SUB_PATH, usecols=["row_id"])
        sub = sample_sub.merge(sub, on="row_id", how="left")
        sub["birds"] = sub["birds"].fillna("nocall")
    return sub

In [ ]:
pred_probas = predict([nets[0]], test_data, names=False)
print(len(pred_probas))
preds = [get_bird_names(get_thresh_preds(pred, thresh=0.1)) for pred in pred_probas]
sub = preds_as_df(data, preds)
print(sub.shape)
sub.to_csv("submission_1_0.csv", index=False)

In [ ]:
pred_probas = predict([nets[1]], test_data, names=False)
print(len(pred_probas))
preds = [get_bird_names(get_thresh_preds(pred, thresh=0.4)) for pred in pred_probas]
sub = preds_as_df(data, preds)
print(sub.shape)
sub.to_csv("submission_1_1.csv", index=False)


In [ ]:
del sub

In [ ]:
gc.collect()

## Filter by location

In [ ]:
from sklearn import preprocessing
lb = preprocessing.MultiLabelBinarizer()
lb = joblib.load("../input/birdclef-for-nadya/labelbinarizer_3.joblib")

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
clf = OneVsRestClassifier(LogisticRegression(random_state=1812, solver="saga", class_weight="balanced", max_iter=100, C=1291.54966501), n_jobs=-1)
clf = joblib.load("../input/birdclef-for-nadya/classifier_3.joblib")

def get_month(x):
    date = x.split("-")
    if len(date) > 1:
        return date[1]
    else:
        return None
df_train["month"] = df_train["date"].astype(str).apply(lambda x: get_month(x))
df_train.dropna(inplace=True)

sc = preprocessing.StandardScaler()
sc.fit(df_train[[ 'latitude', 'longitude','month']].astype(float).to_numpy())

In [ ]:
locs = {}
print(data["site"].unique())
for loc in data["site"].unique():
    with open(Path("../input/birdclef-2021/test_soundscapes")/(loc + "_recording_location.txt"), "r") as f:
        row = {}
        for line in f:
            line = line.strip()
            #print(line, len(line.split(":")))
            if len(line.split(":")) == 2:
                row.update({line.split(":")[0]: line.split(":")[1]})
        
        locs.update({loc: row})
        print(locs)

In [ ]:
data["month"] = data["date"].apply(lambda x: float(str(x)[4:6]))
data['Latitude'] = data["site"].apply(lambda x: locs.get(x).get('Latitude'))
data['Longitude'] = data["site"].apply(lambda x: locs.get(x).get('Longitude'))

for n, row in data.iterrows():
    proba = clf.predict_proba(sc.transform(np.array([row['Latitude'], row['Longitude'], row["month"]]).astype(float).reshape(1,3)))
    data.loc[n, "freq_birds"] = " ".join(lb.classes_[np.nonzero(np.where(proba[0] >= 0.5, proba[0], 0))])
data.to_csv("location.csv")
data.head()

In [ ]:
mod1 = pd.read_csv("submission_1_0.csv")
mod2 = pd.read_csv("submission_1_1.csv")
modpred = mod1.merge(mod2, how="left", on="row_id", suffixes=["_1", "_2"])
modpred["reg_id"] = modpred["row_id"].apply(lambda x: x.split("_")[1])
modpred["audio_id"] = modpred["row_id"].apply(lambda x: x.split("_")[0])
modpred["birds"] = modpred["birds_1"]
modpred = modpred.fillna("nocall")
modpred

In [ ]:
for n, row in modpred[modpred["birds_1"] != modpred["birds_2"]].iterrows():
    if row["birds_1"] == "nocall":
        bird = set(row["birds_2"].split(" "))
    elif row["birds_2"] == "nocall":
        bird = set(row["birds_1"].split(" "))
    else: 
        bird = set(row["birds_2"].split(" ") + row["birds_1"].split(" "))
    #print(bird)
    #print(data[data["id"] == row["audio_id"]]["freq_birds"].values[0])
    freq_birds = set(data[data["id"] == row["audio_id"]]["freq_birds"].values[0].split(" "))
    res = bird & freq_birds
    if len(res) == 0:
        modpred.loc[n, "birds"] = "nocall"
    else:
        modpred.loc[n, "birds"] = " ".join(list(res))
    #print(modpred.loc[n, "birds"])
modpred[["row_id", "birds"]].to_csv("submission_3.csv", index=False)

# Small validation

In [ ]:
new_data = pd.read_csv("submission_3.csv")
new_data['birds'].value_counts()

In [ ]:
def get_metrics(s_true, s_pred):
    s_true = set(s_true.split())
    s_pred = set(s_pred.split())
    n, n_true, n_pred = len(s_true.intersection(s_pred)), len(s_true), len(s_pred)
    
    prec = n/n_pred
    rec = n/n_true
    f1 = 2*prec*rec/(prec + rec) if prec + rec else 0
    
    return {"f1": f1, "prec": prec, "rec": rec, "n_true": n_true, "n_pred": n_pred, "n": n}

In [ ]:
if PUBLIC:
    sub_target = pd.read_csv(TARGET_PATH)
    sub_target = sub_target.merge(pd.read_csv("submission_3.csv"), how="left", on="row_id")
    
    print(sub_target["birds_x"].notnull().sum(), sub_target["birds_x"].notnull().sum())
    assert sub_target["birds_x"].notnull().all()
    assert sub_target["birds_y"].notnull().all()
    
    df_metrics = pd.DataFrame([get_metrics(s_true, s_pred) for s_true, s_pred in zip(sub_target.birds_x, sub_target.birds_y)])
    
    print(df_metrics.mean())

In [ ]:
sub_target[sub_target.birds_y != "nocall"]

In [ ]:
sub_target[sub_target.birds_x != "nocall"]

In [ ]:
for tr in np.linspace(0.01, 1, 50):
    preds = [get_bird_names(get_thresh_preds(pred, thresh=tr)) for pred in pred_probas]
    sub = preds_as_df(data, preds)
    sub_target = pd.read_csv(TARGET_PATH)
    sub_target = sub_target.merge(sub, how="left", on="row_id")
    
    #print(sub_target["birds_x"].notnull().sum(), sub_target["birds_x"].notnull().sum())
    assert sub_target["birds_x"].notnull().all()
    assert sub_target["birds_y"].notnull().all()
    
    df_metrics = pd.DataFrame([get_metrics(s_true, s_pred) for s_true, s_pred in zip(sub_target.birds_x, sub_target.birds_y)])
    print(tr, df_metrics.f1.mean())

### Train model with test data

### Training the model

In [ ]:
def one_step( xb, yb, net, criterion, optimizer, scheduler=None):
  xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        
  optimizer.zero_grad()
  o = net(xb)
  loss = criterion(o, yb)
  loss.backward()
  optimizer.step()
  
  with torch.no_grad():
      l = loss.item()

      o = o.sigmoid()
      yb = (yb > 0.5 )*1.0
      lrap = label_ranking_average_precision_score(yb.cpu().numpy(), o.cpu().numpy())

      o = (o > 0.5)*1.0

      prec = (o*yb).sum()/(1e-6 + o.sum())
      rec = (o*yb).sum()/(1e-6 + yb.sum())
      f1 = 2*prec*rec/(1e-6+prec+rec)

  if  scheduler is not None:
    scheduler.step()

  return l, lrap, f1.item(), rec.item(), prec.item()

In [ ]:
@torch.no_grad()
def evaluate(net, criterion, val_laoder):
    net.eval()

    os, y = [], []
    val_laoder = tqdm(val_laoder, leave = False, total=len(val_laoder))

    for icount, (xb, yb) in  enumerate(val_laoder):

        y.append(yb.to(DEVICE))

        xb = xb.to(DEVICE)
        o = net(xb)

        os.append(o)

    y = torch.cat(y)
    o = torch.cat(os)

    l = criterion(o, y).item()
    
    o = o.sigmoid()
    y = (y > 0.5)*1.0

    lrap = label_ranking_average_precision_score(y.cpu().numpy(), o.cpu().numpy())

    o = (o > 0.5)*1.0

    prec = ((o*y).sum()/(1e-6 + o.sum())).item()
    rec = ((o*y).sum()/(1e-6 + y.sum())).item()
    f1 = 2*prec*rec/(1e-6+prec+rec)

    return l, lrap, f1, rec, prec, 
    

In [ ]:
def one_epoch(net, criterion, optimizer, scheduler, train_laoder, val_laoder):
  net.train()
  l, lrap, prec, rec, f1, icount = 0.,0.,0.,0., 0., 0
  train_laoder = tqdm(train_laoder, leave = False)
  epoch_bar = train_laoder
  
  for (xb, yb) in  epoch_bar:
      #print(len(xb), xb)
      # epoch_bar.set_description("----|----|----|----|---->")
      _l, _lrap, _f1, _rec, _prec = one_step(xb, yb, net, criterion, optimizer)
      l += _l
      lrap += _lrap
      f1 += _f1
      rec += _rec
      prec += _prec

      icount += 1
        
      if hasattr(epoch_bar, "set_postfix") and not icount%10:
          epoch_bar.set_postfix(
            loss="{:.6f}".format(l/icount),
            lrap="{:.3f}".format(lrap/icount),
            prec="{:.3f}".format(prec/icount),
            rec="{:.3f}".format(rec/icount),
            f1="{:.3f}".format(f1/icount),
          )
  
  scheduler.step()

  l /= icount
  lrap /= icount
  f1 /= icount
  rec /= icount
  prec /= icount
  
  l_val, lrap_val, f1_val, rec_val, prec_val = evaluate(net, criterion, val_laoder)
  
  return (l, l_val), (lrap, lrap_val), (f1, f1_val), (rec, rec_val), (prec, prec_val), net


In [ ]:
MODEL_ROOT = Path("models")
class AutoSave:
  def __init__(self, top_k=2, metric="f1", mode="min", root=None, name="ckpt"):
    self.top_k = top_k
    self.logs = []
    self.metric = metric
    self.mode = mode
    self.root = Path(root or MODEL_ROOT)
    assert self.root.exists()
    self.name = name

    self.top_models = []
    self.top_metrics = []

  def log(self, model, metrics):
    metric = metrics[self.metric]
    rank = self.rank(metric)

    self.top_metrics.insert(rank+1, metric)
    if len(self.top_metrics) > self.top_k:
      self.top_metrics.pop(0)

    self.logs.append(metrics)
    self.save(model, metric, rank, metrics["epoch"])


  def save(self, model, metric, rank, epoch):
    t = time.strftime("%Y%m%d%H%M%S")
    name = "{}_epoch_{:02d}_{}_{:.04f}_{}".format(self.name, epoch, self.metric, metric, t)
    name = re.sub(r"[^\w_-]", "", name) + ".pth"
    path = self.root.joinpath(name)

    old_model = None
    self.top_models.insert(rank+1, name)
    if len(self.top_models) > self.top_k:
      old_model = self.root.joinpath(self.top_models[0])
      self.top_models.pop(0)      

    torch.save(model.state_dict(), path.as_posix())

    if old_model is not None:
      old_model.unlink()

    self.to_json()


  def rank(self, val):
    r = -1
    for top_val in self.top_metrics:
      if val <= top_val:
        return r
      r += 1

    return r
  
  def to_json(self):
    # t = time.strftime("%Y%m%d%H%M%S")
    name = "{}_logs".format(self.name)
    name = re.sub(r"[^\w_-]", "", name) + ".json"
    path = self.root.joinpath(name)

    with path.open("w") as f:
      json.dump(self.logs, f, indent=2)

In [ ]:
BATCH_SIZE = 64
NUM_WORKERS = 2

def one_fold(train_set, val_set, epochs=20, save=True, save_root=None):

  save_root = Path(save_root) or MODEL_ROOT

  saver = AutoSave(root=save_root, name=f"birdclef_test_all", metric="f1_val")
  #net = nets[0]
  net = load_net(Path("../input/kkiller-birdclef-models-public/birdclef_resnest50_fold0_epoch_10_f1_val_06471_20210417161101.pth").as_posix())
  net.train() #= get_model(model_name).to(DEVICE)

  criterion = nn.BCEWithLogitsLoss()

  optimizer = optim.Adam(net.parameters(), lr=8e-4, weight_decay = 8e-5)
  scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, eta_min=1e-5, T_max=epochs, verbose=1)

  train_data = TestBirdCLEFDataset(train_set, 
                           sr=SR, duration=DURATION)
  train_laoder = DataLoader(train_data, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True, pin_memory=True)

  val_data = TestBirdCLEFDataset(train_set,  sr=SR, duration=DURATION)
  val_laoder = DataLoader(val_data, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)

  epochs_bar = tqdm(list(range(epochs)), leave=False)
  print(epochs)
  for epoch  in epochs_bar:
    epochs_bar.set_description(f"--> [EPOCH {epoch:02d}]")
    #net.train()

    (l, l_val), (lrap, lrap_val), (f1, f1_val), (rec, rec_val), (prec, prec_val), net = one_epoch(
        net=net,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        train_laoder=train_laoder,
        val_laoder=val_laoder,
      )

    epochs_bar.set_postfix(
    loss="({:.6f}, {:.6f})".format(l, l_val),
    prec="({:.3f}, {:.3f})".format(prec, prec_val),
    rec="({:.3f}, {:.3f})".format(rec, rec_val),
    f1="({:.3f}, {:.3f})".format(f1, f1_val),
    lrap="({:.3f}, {:.3f})".format(lrap, lrap_val),
    )

    print(
        "[{epoch:02d}] loss: {loss} lrap: {lrap} f1: {f1} rec: {rec} prec: {prec}".format(
            epoch=epoch,
            loss="({:.6f}, {:.6f})".format(l, l_val),
            prec="({:.3f}, {:.3f})".format(prec, prec_val),
            rec="({:.3f}, {:.3f})".format(rec, rec_val),
            f1="({:.3f}, {:.3f})".format(f1, f1_val),
            lrap="({:.3f}, {:.3f})".format(lrap, lrap_val),
        )
    )

    if save:
      metrics = {
          "loss": l, "lrap": lrap, "f1": f1, "rec": rec, "prec": prec,
          "loss_val": l_val, "lrap_val": lrap_val, "f1_val": f1_val, "rec_val": rec_val, "prec_val": prec_val,
          "epoch": epoch,
      }

      saver.log(net, metrics)
  return net

In [ ]:
pd.read_csv(TARGET_PATH).head()

In [ ]:
from sklearn.model_selection import train_test_split

second_data = pd.read_csv("submission_3.csv")
second_data["audio_id"] = second_data["row_id"].apply(lambda x: x.split("_")[0])
second_data["site"] = second_data["row_id"].apply(lambda x: x.split("_")[1])
second_data["seconds"] = second_data["row_id"].apply(lambda x: x.split("_")[2])
second_data = pd.concat([second_data, pd.read_csv(TARGET_PATH)], axis=0)
second_data = second_data[second_data["birds"] != "nocall"].reset_index()
print(second_data.head())

def train(train_set, epochs=2, save=True, n_splits=5, seed=1812, save_root=None, suffix="", folds=None):
    gc.collect()
    torch.cuda.empty_cache()

    save_root = save_root or MODEL_ROOT
    save_root.mkdir(exist_ok=True, parents=True)

    
    train_idx, val_idx = train_test_split(np.arange(len(train_set.index)), test_size=0.2, random_state=seed)

    nets2 = []
    for fold in folds:
        nets2.append(one_fold(train_set=train_set.loc[train_idx].reset_index() , val_set=train_set.loc[val_idx].reset_index(), epochs=epochs, save=save, save_root=save_root))

    gc.collect()
    torch.cuda.empty_cache()
    return nets2

In [ ]:
class TestBirdCLEFDataset(Dataset):
    def __init__(self, data, sr=SR, n_mels=128, fmin=0, fmax=None, duration=DURATION, step=None, res_type="kaiser_fast", resample=True):
        
        self.data = data
        
        self.sr = sr
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax or self.sr//2

        self.duration = duration
        self.audio_length = self.duration*self.sr
        self.step = step or self.audio_length
        
        self.res_type = res_type
        self.resample = resample

        self.mel_spec_computer = MelSpecComputer(sr=self.sr, n_mels=self.n_mels, fmin=self.fmin,
                                                 fmax=self.fmax)
    def __len__(self):
        return len(self.data)

    def read_ready_file(self, filename):
        audios = []
        fname = filename + ".npy"
        melspec = np.load(TARGET_AUDIO_1/fname)
        image = np.nan_to_num(normalize(melspec)) 
        
        image = mono_to_color(image)
        image = apply_augmentation(torch.Tensor(image))
        #print(fname)
        audios.append(image)
        return image
            
    def __getitem__(self, idx):
        labels = np.zeros(len(LABEL_IDS))
        for label in self.data.loc[idx, "birds"].split(" "):
            labels[LABEL_IDS[label]] = 1
        return self.read_ready_file(self.data.loc[idx, "row_id"]), labels

In [ ]:
nets2 = train(second_data, epochs=3, suffix=f"_sr{SR}_d{DURATION}_v2405_v1", folds=range(5))

In [ ]:
test_data = BirdCLEFDataset(data=data, transform = True)
pred_probas = []
for n in range(10):
    pred_probas.append(predict(nets2, test_data, names=False))
print(len(pred_probas))
pred_probas = np.stack(pred_probas, axis=0).mean(axis=0)

In [ ]:
preds = [get_bird_names(get_thresh_preds(pred, thresh=0.3)) for pred in pred_probas]
sub = preds_as_df(data, preds)
print(sub.shape)
sub

In [ ]:
sub.to_csv("submission.csv", index=False)

In [ ]:
if TARGET_PATH:
    sub_target = pd.read_csv(TARGET_PATH)
    sub_target = sub_target.merge(pd.read_csv("submission.csv"), how="left", on="row_id")
    
    print(sub_target["birds_x"].notnull().sum(), sub_target["birds_x"].notnull().sum())
    assert sub_target["birds_x"].notnull().all()
    assert sub_target["birds_y"].notnull().all()
    
    df_metrics = pd.DataFrame([get_metrics(s_true, s_pred) for s_true, s_pred in zip(sub_target.birds_x, sub_target.birds_y)])
    
    print(df_metrics.mean())

In [ ]:
!rm -rf test_data_1 test_data_2 